In [1]:
source_path = '../../client/src/datasets/assets-greensboro.json'

In [2]:
import json
assets = json.load(open(source_path))

In [3]:
assets[0]

{'childIds': ['aiQ2'],
 'geometry': {'coordinates': [-79.65685579971908, 36.17972957289909],
  'type': 'Point'},
 'id': 'aqCS',
 'location': [-79.65685579971908, 36.17972957289909],
 'name': 'Pole 1',
 'parentIds': ['abH0', 'aiuT'],
 'typeId': 'p'}

In [4]:
asset_by_id = {_['id']: _ for _ in assets}

In [5]:
def make_point_geojson(xy):
    return {
        'type': 'Point',
        'coordinates': xy,
    }

def make_line_geojson(xys):
    return {
        'type': 'LineString',
        'coordinates': xys,
    }

In [6]:
for asset in assets:
    if 'location' in asset:
        xy = asset['location']
        asset['geometry'] = make_point_geojson(xy)
    elif asset['typeId'] == 'l':
        pole_assets = [asset_by_id[_] for _ in asset['childIds']]
        xys = [_['location'] for _ in pole_assets]
        asset['geometry'] = make_line_geojson(xys)
    elif asset['parentIds']:
        for parent_id in asset['parentIds']:
            parent_asset = asset_by_id[parent_id]
            if 'location' in parent_asset:
                xy = parent_asset['location']
                asset['geometry'] = make_point_geojson(xy)
                break
    if 'wkt' in asset:
        del asset['wkt']

In [7]:
from shapely.geometry import shape
gs = []
for asset in assets:
    try:
        g = shape(asset['geometry'])
    except KeyError:
        print(asset)
    else:
        gs.append(g)

{'id': 'abkC', 'typeId': 'xc', 'name': 'Recloser 9', 'parentIds': [], 'connectedIds': ['ahwI', 'ahwS', 'ahwT', 'akM1', 'abku', 'abkv', 'abkw', 'abkz', 'abkA', 'abkB', 'abkD', 'abkH', 'abkI', 'abkK', 'abkL', 'abkN', 'abkO']}


In [8]:
from shapely.geometry import GeometryCollection
gc = GeometryCollection(gs)

In [9]:
gc.bounds

(-79.71827225092947, 36.13830085783994, -79.49956870558653, 36.30469203650488)

In [10]:
gc.centroid.coords[0]

(-79.60975239966439, 36.20116405881486)

In [11]:
values = set()
for asset in assets:
    if asset['typeId'] == 'l':
        values.add(asset['KV'])
values

{0.42, 12.47, 69.0}

In [12]:
values = set()
for asset in assets:
    if asset['typeId'] == 'm':
        values.add(asset['KV'])
values

{0.42, 12.47}

In [18]:
values = set()
for asset in assets:
    if asset['typeId'] == 'm':
        values.add(asset['peakKW'])
values

{0.76, 1.66, 2.74, 16.11, 38.25, 40.74, 139.55}

In [17]:
asset

{'KV': 0.42,
 'connectedIds': ['abP2'],
 'geometry': {'coordinates': [-79.63002659266567, 36.2809205913053],
  'type': 'Point'},
 'id': 'akM2',
 'location': [-79.63002659266567, 36.2809205913053],
 'name': 'Meter 1',
 'parentIds': ['aqM3'],
 'peakKW': 0.76,
 'typeId': 'm',
 'yearlyKWH': 4147.0}

In [13]:
for asset in assets:
    if asset['typeId'] == 'st':
        asset['outKV'] = 69.0

In [14]:
values = set()
for asset in assets:
    if asset['typeId'].startswith('s'):
        values.add(asset['outKV'])
        asset['KV'] = asset['outKV']
values

{12.47, 69.0}

In [15]:
json.dump(assets, open(source_path, 'wt'), indent=2)